# Financial News Agent - Gemini (gemini-2.0-flash-lite-001)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install feedparser
!pip install -q google-generativeai PyMuPDF sentence-transformers faiss-cpu gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=36994058193a4552d2f2bb0afe5d150ebe01300e2a94374449069565e3b914fc
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 35.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 64.3 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:

### Import the required libraries

In [3]:
# Libraries for Setup & Configuration
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient
# Libraries for Feed Parsing and Typing
import feedparser
import urllib.parse
from typing import List
# Libraries for load, save and update memory 
import json
from pathlib import Path
# Libraries for summarizing and plotting the confidence trend
import pandas as pd
import matplotlib.pyplot as plt
# Libraries for UI 
import gradio as gr

### Configure Gemini with secure API key (from Kaggle Secrets)

In [4]:
genai.configure(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY")) # Key value is configured in Add-ons/Secrets

### Load a lightweight Gemini model suitable for quick responses

In [5]:
model = genai.GenerativeModel("gemini-2.0-flash-lite-001") # Model was selected based on the rate limits of the Gemini API

In [6]:
# Agent Memory: In-memory conversation history (used for context & memory simulation)
conversation_history = []

# Agent Functions

def rewrite_to_query(user_prompt: str) -> str:
    prompt = f"Extract short search keywords from: \"{user_prompt}\""
    return model.generate_content(prompt).text.strip().replace(" ", "+")


def fetch_articles(user_prompt, max_results=5):
    #Clean the query (remove newlines, extra spaces)
    clean_prompt = " ".join(user_prompt.strip().split())
    # URL encode
    encoded_query = urllib.parse.quote_plus(clean_prompt)
    # Build Google News RSS URL
    rss_url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-IN&gl=IN&ceid=IN:en"
    # Parse feed
    feed = feedparser.parse(rss_url)
    # Return top N articles
    return feed.entries[:max_results]


def summarize_with_context(articles: List[dict], prompt: str, memory: str = "") -> List[dict]:
    summarized = []
    for article in articles:
        text = f"""
User prompt: {prompt}
Context: {memory}

News:
Title: {article['title']}
Summary: {article['summary']}

Give 3 bullet-point insights relevant to user query. Cite link: {article['link']}
"""
        summary = model.generate_content(text).text
        summarized.append({"title": article['title'], "summary": summary, "link": article['link']})
    return summarized


def generate_plan(summaries: List[dict], prompt: str, memory: str) -> str:
    text = f"""
User Query: {prompt}
Context: {memory}
Summarized News:
{''.join([s['summary'] for s in summaries])}

Give output as function: investment_plan(decision, reasons, confidence_score)
"""
    return model.generate_content(text).text.strip()


def evaluate_plan(plan: str) -> str:
    text = f"""
Evaluate this investment_plan:
{plan}

Rate 1-10 with reasoning.
"""
    return model.generate_content(text).text.strip()


# Master Agent Function (Loop-Aware)

def financial_agent(user_prompt: str, rag_context: str = ""):
    # Add to memory
    conversation_history.append({"role": "user", "content": user_prompt})
    context_memory = " ".join([item["content"] for item in conversation_history if item["role"] == "user"])

    # Convert to search query
    query = rewrite_to_query(user_prompt)

    # Fetch news
    articles = fetch_articles(query)

    # Summarize with memory
    summaries = summarize_with_context(articles, user_prompt, context_memory)

    # Action plan
    action_plan = generate_plan(summaries, user_prompt, rag_context)

    # Evaluation
    evaluation = evaluate_plan(action_plan)

    # Add system-generated context to memory
    conversation_history.append({"role": "agent", "content": action_plan})

    # Return response
    return {
        "query": query,
        "summaries": summaries,
        "action_plan": action_plan,
        "evaluation": evaluation
    }

In [7]:
MEMORY_FILE = Path("agent_memory.json")

def load_memory():
    if MEMORY_FILE.exists():
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    return []

def save_memory(memory):
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f, indent=2)

# On initialization
conversation_history = load_memory()

# At end of agent run
def update_memory(user_prompt, agent_response):
    conversation_history.append({"role": "user", "content": user_prompt})
    conversation_history.append({"role": "agent", "content": agent_response})
    save_memory(conversation_history)

In [8]:
def summarize_to_dataframe(summaries):
    data = []
    for s in summaries:
        data.append({
            "Title": s["title"],
            "Summary": s["summary"],
            "Link": s["link"]
        })
    return pd.DataFrame(data)


def plot_confidence_trend(memory):
    scores = []
    labels = []
    for item in memory:
        if item["role"] == "agent":
            content = item["content"]
            try:
                score = int([s for s in content.split() if s.isdigit()][-1])
                scores.append(score)
                labels.append(len(scores))
            except:
                continue
    if not scores:
        return None
    plt.figure(figsize=(6,3))
    plt.plot(labels, scores, marker='o')
    plt.title("Confidence Scores Over Time")
    plt.xlabel("Query Number")
    plt.ylabel("Confidence Score")
    plt.grid(True)
    return plt

In [11]:
def run_agent_interface(user_prompt, rag_context):
    result = financial_agent(user_prompt, rag_context)

    # Save to persistent memory
    update_memory(user_prompt, result["action_plan"])

    # Format output
    df = summarize_to_dataframe(result["summaries"])
    plot = plot_confidence_trend(conversation_history)

    return result["action_plan"], result["evaluation"], df, plot


with gr.Blocks() as demo:
    gr.Markdown("## 📈 Financial News Agent with Gemini")

    user_prompt = gr.Textbox(label="Your Prompt", placeholder="e.g. Give me the highlights of today's market")
    rag_input = gr.Textbox(label="Domain Context (Optional)", placeholder="e.g. Top Q1 earners")

    run_button = gr.Button("Analyze")

    with gr.Row():
        action_output = gr.Textbox(label="📊 Investment Plan")
        eval_output = gr.Textbox(label="🧪 Evaluation")

    df_output = gr.Dataframe(label="Summarized News")
    chart_output = gr.Plot(label="Confidence Trend")

    run_button.click(fn=run_agent_interface, 
                     inputs=[user_prompt, rag_input],
                     outputs=[action_output, eval_output, df_output, chart_output])

In [12]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://a715dffaca2b14377d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
